Based on https://github.com/Qiskit/qiskit-tutorials/blob/master/legacy_tutorials/ignis/6a_state_tomography.ipynb?fbclid=IwAR1Pf1nH2CBPUWDdsa_xBiYs1kYDu8hwI9dVvr63z7ypmhy2UlR6QpMgJWc, 2-Qubit Conditional State Tomography

# Verifying QEC with Two Extra Qubits on IBM Q computers

- Code References
+ [Qiskit Tutorial, 2-Qubit Conditional State Tomography](https://github.com/Qiskit/qiskit-tutorials/blob/master/legacy_tutorials/ignis/6a_state_tomography.ipynb?fbclid=IwAR1Pf1nH2CBPUWDdsa_xBiYs1kYDu8hwI9dVvr63z7ypmhy2UlR6QpMgJWc)
+ [Qiskit source code](https://github.com/Qiskit) `Note` Some functions are modified from built-in methods.

## Initialization

In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit import QuantumRegister, ClassicalRegister
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from qiskit.circuit import Parameter
from qiskit.result import Result

import qiskit.ignis.verification.tomography as tom
from qiskit.providers.ibmq.managed import IBMQJobManager
from qiskit.providers.jobstatus import JOB_FINAL_STATES
from qiskit.quantum_info import state_fidelity
from qiskit.providers.aer import noise

# Importing other packages
import matplotlib.pyplot as plt
import pickle
import numpy as np
from copy import deepcopy
import time
# Importing measurement calibration functions
from qiskit.ignis.mitigation.measurement import (complete_meas_cal, tensored_meas_cal,
                                                 CompleteMeasFitter, TensoredMeasFitter)
from itertools import product

# Importing tomography fitter functions
from qiskit.ignis.verification.tomography.fitters import lstsq_fit
from qiskit.ignis.verification.tomography.basis import TomographyBasis
from ast import literal_eval
import itertools as it
from qiskit.ignis.verification.tomography.data import combine_counts, count_keys

#Visualization
%qiskit_job_watcher
%qiskit_progress_bar
from qiskit.tools.parallel import parallel_map
parallel_map(np.sin, np.linspace(0,10,100));

Accordion(children=(VBox(layout=Layout(max_width='710px', min_width='710px')),), layout=Layout(max_height='500…

<IPython.core.display.Javascript object>

In [2]:
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

In [3]:
#Getting backends
backend=provider.get_backend('ibmq_16_melbourne')
backend_sim=provider.get_backend('ibmq_qasm_simulator')
backend_sim_local=Aer.get_backend('qasm_simulator')
backend_sv=Aer.get_backend('statevector_simulator')

In [4]:
data_list=[0,1,2,3,4]
syn=[5]
flag=[6]

## State Preparation

In [136]:
"""
StatePreparation(stabilizer: int)
    Return one of the three stabilizers for state preparation in SM of QEC with two extra qubits.
    Variable:
        stabilizer: 0,1, or 2
    Return:
        q: QuantumRegister(7)
        qc: QuantumCircuit(q)

q, qc=StatePreparation(0)
qc.draw()
"""
def StatePreparation(stabilizer=0):#if stabilizer=-1 or any value that is not 0,1,2, will return a circuit without stabilizer
    q = QuantumRegister(7)
    qc = QuantumCircuit(q)
    #prepare the 5-qubit graph state
    qc.h(q[0])
    qc.h(q[1])
    qc.h(q[2])
    qc.h(q[3])
    qc.h(q[4])
    qc.cz(q[0],q[1])
    qc.cz(q[2],q[3])
    qc.cz(q[1],q[2])
    qc.cz(q[3],q[4])
    qc.cz(q[0],q[4])
    #syndrome and flag extraction
    #stabilizer order: from left to right, Fig 2(a), SM of two extra qubits
    if(stabilizer==0):
        #first stabilizer 
        qc.h(q[5])
        qc.cx(q[5],q[0])
        qc.cx(q[5],q[6])
        qc.cz(q[1],q[5])
        qc.cx(q[5],q[6])
        qc.cz(q[4],q[5])
        qc.h(q[5])
    elif(stabilizer==1):
    #second stabilizer
        qc.h(q[5])
        qc.cz(q[0],q[5])
        qc.cx(q[5],q[6])
        qc.cx(q[5],q[1])
        qc.cx(q[5],q[6])
        qc.cz(q[2],q[5])
        qc.h(q[5])
    elif(stabilizer==2):
        #third stabilizer
        qc.h(q[5])
        qc.cx(q[5],q[3])
        qc.cx(q[5],q[6])
        qc.cz(q[2],q[5])
        qc.cx(q[5],q[6])
        qc.cz(q[4],q[5])
        qc.h(q[5])
    #measure syndrome and flag & do state tomography for the data qubits
    qc.barrier()
    #qc.measure(q[5],c[5])
    #qc.measure(q[6],c[6])
    return q, qc

In [6]:
"""
reduced(v: ndarray, sf: str)
    Return the normalized projection result of given state vector onto certain subspace of syndrome&flag outcomes.
    Variables:
        v: ndarray, length 2**7
        sf: '00', '01', '10', or '11', the first bit represents syndrome outcome and the second represents flag.
    Return:
        ndarray (2**5, 1)

    we should have projected the ideal final state to the final state, ...
    'reduced(v,sf)' returns the projected states based on 'sf'.
    At the moment, for the (ideal) simulation result, 'sf' can only be '00'
    and the func simply returns the first 32 elements of v.
"""
def reduced(v, sf):
    s=int(sf[0])
    f=int(sf[1])
    out=s*2+f
    res=[]
    for i in range(v.shape[0]):
        if((i>>5)==out):#((i%4)==out):#
            res.append(v[i])
    res=np.array(res)
    res=res/np.sqrt(np.sum(np.power(np.abs(res),2)))
    return np.array(res)

## Pre-processing

### Error Mitigation

In [7]:
def msmtErrorMitFilter(backend, measured_qubit_list):
    # backend is the backend from which we obtain measurement error rates
    #measured_qubit_list contains
    qr = QuantumRegister(len(measured_qubit_list))
    qubit_list = range(len(measured_qubit_list))
    
    # Create the measurement calibration circuits
    meas_calibs, state_labels = complete_meas_cal(qubit_list=qubit_list, qr=qr, circlabel='mcal')
    
    # Create a noise model with the qubits being measured
    noise_model = noise.NoiseModel()
    back_prop = backend.properties()
    for qi in range(len(measured_qubit_list)):
        m0_p1 = back_prop.qubit_property(qi,name='prob_meas0_prep1')[0]
        m1_p0 = back_prop.qubit_property(qi,name='prob_meas1_prep0')[0]
        read_err = noise.errors.readout_error.ReadoutError([[1-m1_p0, m1_p0],[m0_p1,1-m0_p1]])
        noise_model.add_readout_error(read_err, [qi])
    
    # Execute the calibration circuits
    job = execute(meas_calibs, backend=backend_sim_local, shots=1000, noise_model=noise_model)
    cal_results = job.result()
    
    # Calculate the calibration matrix with the noise model
    meas_fitter = CompleteMeasFitter(cal_results, state_labels, qubit_list=qubit_list, circlabel='mcal')

    # Get the filter object
    meas_filter = meas_fitter.filter
    
    return meas_filter

# Please check msmtErrorMitOnTom().

Changes: 
    - `old_counts` -> list of results.get_counts(data_idx)
    - `mitigated_results` is `results` instead of a deepcopy of `results`
    - For results obtained from melbourne, will return a dict like {"(X,X,X,X,X)": {'00 10101':12.2343234.... instead

In [96]:
def msmtErrorMitOnTom(results, meas_filter, backend_type='simulation', basis_list=None):
    
#     print(results)
    #mitigated_results = deepcopy(results)
    old_counts=[]
    for data_idx in range(243):#, _ in enumerate(results.results):
        old_counts.append(results.get_counts(data_idx))
    if(backend_type=='simulation'):
        mitigated_results = results
    else:
        mitigated_results = {}
    for data_idx in range(243):#results.results):
        new_counts = {}
        # removes key structure
        for reg_key in old_counts[data_idx]:
            reg_bits = reg_key.split(' ')
            reg_bits = reg_bits[0]+reg_bits[1]
            new_counts[reg_bits]=old_counts[data_idx][reg_key]
                        
        # applies measurement filter
#         print(new_counts)
        new_counts1 = meas_filter.apply(new_counts)
#        print(type(new_counts1))
#        print(new_counts1)
        
#         #replaces key structure
        new_counts2 = {}
        for reg_key in new_counts1:
            #print(type(reg_key))
            reg_bits = []
            reg_bits.append(reg_key[0:2])
            reg_bits.append(reg_key[2:7])
            reg = reg_bits[0] + " " + reg_bits[1]
            new_counts2[reg]=new_counts1[reg_key]
        #print(new_counts2)
        if(backend_type=='simulation'):
            mitigated_results.results[data_idx].data.counts = \
                mitigated_results.results[data_idx]. \
                data.counts.from_dict(new_counts1)
        elif(backend_type=='experiment'):
            mitigated_results[basis_list[data_idx]] = \
                new_counts2
#         new_result.results[resultidx].data.counts = \
#                     new_result.results[resultidx]. \
#                     data.counts.from_dict(new_counts)
        
#     print(mitigated_results)
    return mitigated_results

In [9]:
def sort_list(list1, list2): 
    # sorts list 1 based on the sort indices of list 2
    zipped_pairs = zip(list2, list1) 
    z = [x for _, x in sorted(zipped_pairs)] 
      
    return z 

In [10]:
def extractMsmtQubits(circuit):

    # 'circuit' is the circuit for which we extract the list of measurement qubits
    # This is some messy code. Not sure if we necessarily have to parse the qasm file of a circuit to find the qubits being measured
    # Doesn't work too well when there are two classical registers. I've written up a fix but not happy with it.
    qasm_obj = circuit.qasm()
    t = qasm_obj.find("measure")
    qasm_obj = qasm_obj[t:-1]
    qasm_obj = qasm_obj.split(' ')
    listmsmt = []
    listclass = []
    for _ in qasm_obj:
        if 'q' in _:
            number = []
            for j in range(len(_)):
                if _[j] =='[':
                    temp1 = _[j+1:j+3].strip(']')
            listmsmt.append(int(temp1))
    for _ in qasm_obj:
        if 'c' in _:
            number = []
            for j in range(len(_)):
                if _[j] =='[':
                    temp1 = _[j+1:j+3].strip(']')
            if temp1 == '0' and len(listclass) > 4:
                temp1 = '5'
            if temp1 == '1' and len(listclass) > 4:
                temp1 = '6'
            listclass.append(int(temp1))
    listmsmt = sort_list(listmsmt, listclass)
#     print(listmsmt)
#     print(listclass)
    return listmsmt

### Executing the Circuits

Circuit | Description 
---| ---
qc0 | As shown in [6], state prepartion + stabilizer, no measurement
qc1 | add measurements on syndrome and flag qubits
qctom | add measurements on syndrome and flag qubits, then do state tomography on data qubits, used for tomography by both simulation and experiment
qctom_no_anc | qctom without measurement, used for 'tom.stateTomographyFitter'


In [109]:
"""
executeStatePreparation(stabilizer: int, backend_type: str, mitigation: boolean)
Variables:
    stabilizer: 0, 1 or 2, determine which stabilizer to be chosen
    backend_type: 'simulation', use qasm to simulate the circuit or 'experiment', use melbourne to run real quantum experiement
    mitigation: True, use error mitigation to preprocess the results; False: no error mitigation.
    
Return:
    backend_type=='simulation':
        psi_qc, qctom, qctom_no_anc, results_sim
"""
def executeStatePreparation(stabilizer=0, backend_type='simulation', mitigation=True):
    #state vector
    q0, qc0=StatePreparation(stabilizer)
    qc0.draw()
    
    qc1=deepcopy(qc0)
    ca1 = ClassicalRegister(2)
    qc1.add_register(ca1)
    qc1.measure([q0[5],q0[6]],[ca1[0],ca1[1]])
    job = execute(qc1, backend_sv)
    psi_qc = job.result().get_statevector(qc1)
    
    #qasm simulation
    qctom = tom.state_tomography_circuits(qc0, data_list, meas_labels='Pauli', meas_basis='Pauli')
    qctom_no_anc = deepcopy(qctom)
    for qc in qctom:
        ca = ClassicalRegister(2)
        qc.add_register(ca)
        qc.measure([q0[5],q0[6]],[ca[0],ca[1]])
        
    # Create a noise model with the qubits being measured ------
    # ------used for testing measurement error mitigation on the simulator
    if(backend_type=='simulation'):
        noise_model = noise.NoiseModel()
        back_prop = backend.properties()
        measured_qubit_list = extractMsmtQubits(qctom[-1])
        for qi in range(len(measured_qubit_list)):
            m0_p1 = back_prop.qubit_property(qi,name='prob_meas0_prep1')[0]
            m1_p0 = back_prop.qubit_property(qi,name='prob_meas1_prep0')[0]
            read_err = noise.errors.readout_error.ReadoutError([[1-m1_p0, m1_p0],[m0_p1,1-m0_p1]])
            noise_model.add_readout_error(read_err, [qi])

        job = execute(qctom, backend=backend_sim, optimization_level=0, shots=8192, noise_model=noise_model)
        results_sim=job.result()
        # Create an error mitigation filter ------
        # ------used for testing measurement error mitigation on the simulator
        if(mitigation==True):
            measured_qubit_list = extractMsmtQubits(qctom[-1]) # assuming all the tomo circuits are identical except for basis changes
            #Print(measured_qubit_list)
            filt = msmtErrorMitFilter(backend, measured_qubit_list)
            # Perform error mitigation on all the results
            results_sim_MM = msmtErrorMitOnTom(results_sim, filt)
            results_sim = results_sim_MM

        return (psi_qc, qctom, qctom_no_anc, results_sim)#, tomo_qc_sim)
    
    if(backend_type=='experiment'):
        max_job_count = 5
        max_exp_per_job = 54

        qctom = transpile(qctom, backend=backend)

        #results_real=[]
        #tomo_qc_real=[]

        job_manager = IBMQJobManager()
        start_time = time.time()
        #for qc in qctom:
        job_set_foo = job_manager.run(qctom, backend=backend, name='qctom', shots=8192, max_experiments_per_job=54)
        job_status = job_set_foo.statuses()
        while job_status[max_job_count-1] not in JOB_FINAL_STATES:
            print(f'Status @ {time.time()-start_time:0.0f} s: {job_status[max_job_count-1]},'
               f' ')
            time.sleep(10)
            job_status = job_set_foo.statuses()

        results_real = job_set_foo.results()   

        # Create an error mitigation filter
        if(mitigation==True):
            measured_qubit_list = extractMsmtQubits(qctom_real[-1]) # assuming all the tomo circuits are identical except for basis changes
            filt = msmtErrorMitFilter(backend, measured_qubit_list)
            # Perform error mitigation on all the results
            results_real_MM = msmtErrorMitOnTom(res_real, filt, backend_type='experiment', basis_list=basis)
            results_real=results_real_MM
            #results_real will be a dict
        return (psi_qc, qctom, qctom_no_anc, results_real)
    
    return (psi_qc, qctom, qctom_no_anc, None)

In [63]:
sim_dict={}
for i in range(1):
    data = executeStatePreparation(i, mitigation=True)
    sim_dict[i]=data
    #pickle.dump(data, open("Stabilizer%d"%i, "wb"))


In [137]:
real_dict[-1]=executeStatePreparation(stabilizer=-1, backend_type='experiment', mitigation=True)

Status @ 0 s: JobStatus.INITIALIZING, 
Status @ 12 s: JobStatus.VALIDATING, 
Status @ 23 s: JobStatus.RUNNING, 
Status @ 35 s: JobStatus.RUNNING, 
Status @ 46 s: JobStatus.RUNNING, 
Status @ 58 s: JobStatus.RUNNING, 
Status @ 70 s: JobStatus.RUNNING, 
Status @ 82 s: JobStatus.RUNNING, 
Status @ 93 s: JobStatus.RUNNING, 
Status @ 105 s: JobStatus.RUNNING, 
Status @ 117 s: JobStatus.RUNNING, 
Status @ 129 s: JobStatus.RUNNING, 
Status @ 141 s: JobStatus.RUNNING, 
Status @ 152 s: JobStatus.RUNNING, 
Status @ 163 s: JobStatus.RUNNING, 
Status @ 175 s: JobStatus.RUNNING, 
Status @ 190 s: JobStatus.RUNNING, 
Status @ 201 s: JobStatus.RUNNING, 
Status @ 213 s: JobStatus.RUNNING, 
Status @ 224 s: JobStatus.RUNNING, 
Status @ 236 s: JobStatus.RUNNING, 
Status @ 252 s: JobStatus.RUNNING, 
Status @ 264 s: JobStatus.RUNNING, 
Status @ 275 s: JobStatus.RUNNING, 
Status @ 287 s: JobStatus.RUNNING, 
Status @ 299 s: JobStatus.RUNNING, 
Status @ 310 s: JobStatus.RUNNING, 
Status @ 322 s: JobStatus.RUNN

In [ ]:
real_dict={}
for Round in range(5):
    for i in [-1,0,1,2]:
        data = executeStatePreparation(stabilizer=i, backend_type='experiment', mitigation=True)
        real_dict[(i, Round)]=data
        pickle.dump(data, open("stabilizer %d round %d.pickle") % (i, Round), "wb")
        

In [ ]:
pickle.dump(real_dict[-1], open('test.pickle', "wb"))

In [110]:
real_dict={}
for i in range(1):
    data = executeStatePreparation(stabilizer=i, backend_type='experiment', mitigation=True)
    real_dict[i]=data

Status @ 0 s: JobStatus.INITIALIZING, 
Status @ 10 s: JobStatus.INITIALIZING, 
Status @ 20 s: JobStatus.INITIALIZING, 
Status @ 32 s: JobStatus.INITIALIZING, 
Status @ 44 s: JobStatus.RUNNING, 
Status @ 55 s: JobStatus.RUNNING, 
Status @ 67 s: JobStatus.RUNNING, 
Status @ 78 s: JobStatus.RUNNING, 
Status @ 91 s: JobStatus.RUNNING, 
Status @ 102 s: JobStatus.RUNNING, 
Status @ 114 s: JobStatus.RUNNING, 
Status @ 125 s: JobStatus.RUNNING, 
Status @ 137 s: JobStatus.RUNNING, 
Status @ 149 s: JobStatus.RUNNING, 
Status @ 161 s: JobStatus.RUNNING, 
Status @ 173 s: JobStatus.RUNNING, 
Status @ 185 s: JobStatus.RUNNING, 
Status @ 197 s: JobStatus.RUNNING, 
Status @ 209 s: JobStatus.RUNNING, 
Status @ 220 s: JobStatus.RUNNING, 
Status @ 232 s: JobStatus.RUNNING, 
Status @ 244 s: JobStatus.RUNNING, 
Status @ 256 s: JobStatus.RUNNING, 
Status @ 268 s: JobStatus.RUNNING, 
Status @ 279 s: JobStatus.RUNNING, 
Status @ 291 s: JobStatus.RUNNING, 
Status @ 303 s: JobStatus.RUNNING, 
Status @ 315 s: Jo

In [127]:
def append(res, new_res, backend_type='simulation', basis_list=None):
    #for experiments, data and new_data should either 
    keys=count_keys(7)
    new_keys=[]
    for key in keys:
        new_keys.append(key[:2]+" "+key[2:])
    if(backend_type=='simulation'):
        for resultidx in range(243):
            old_counts=res.get_counts([resultidx])
            new_counts=new_res.get_counts([resultidx])
            sum_counts={}
            for key in new_keys:#res_sim.results[0].data.counts
                ncnt=new_counts.get(key, 0)
                ocnt=old_counts.get(key, 0)
                if(ncnt+ocnt>0):
                    sum_counts[key]=ncnt+ocnt

            res.results[resultidx].data.counts = \
                res.results[resultidx]. \
                data.counts.from_dict(sum_counts)

        return res
    
    if(backend_type=='experiment'):
        #Whatever the types of res and new_res are, we will always return a dict in this form: {"(X,X,X,X,X)": {"00 11010":0.23234...
        if(type(res)!=dict):
            old_counts=[]
            for i in range(243):
                old_counts.append(res.get_counts(i))
        else:
            old_counts=res

        if(type(new_res)!=dict):
            new_counts=[]
            for i in range(243):
                new_counts.append(new_res.get_counts(i))
        else:
            new_counts=new_res
            
        for i in range(243):
            for key in new_keys:
                if(new_counts[basis_list[i]].get(key,0)!=0):
                    if(old_counts[basis_list[i]].get(key, 0)==0):
                        old_counts[basis_list[i]][key]=new_counts[basis_list[i]][key]
                    else:
                        old_counts[basis_list[i]][key]=old_counts[basis_list[i]][key] + new_counts[basis_list[i]][key]
        return old_counts
    

### Pre-processing

In [14]:
def bin_format(integer, length):
    return f'{integer:0>{length}b}'
              
def measurement_matrix(label, outcome):
    if(label=='X'):
        if(outcome=='0' or outcome==0):
            return np.array([[0.5+0.j, 0.5+0.j], [0.5+0.j, 0.5+0.j]], dtype=complex)
        elif(outcome=='1' or outcome==1):
            return np.array([[ 0.5+0.j, -0.5+0.j], [-0.5+0.j,  0.5+0.j]], dtype=complex)
    elif(label=='Y'):
        if(outcome=='0' or outcome==0):
            return np.array([[ 0.5+0.j, -0. -0.5j], [ 0. +0.5j,  0.5+0.j ]], dtype=complex)
        elif(outcome=='1' or outcome==1):
            return np.array([[ 0.5+0.j, 0. +0.5j], [-0. -0.5j,  0.5+0.j ]], dtype=complex)
    elif(label=='Z'):
        if(outcome=='0' or outcome==0):
            return np.array([[1.+0.j, 0.+0.j], [0.+0.j, 0.+0.j]], dtype=complex)
        elif(outcome=='1' or outcome==1):
            return np.array([[0.+0.j, 0.+0.j], [0.+0.j, 1.+0.j]], dtype=complex)

def _binomial_weights(counts, beta):
    shots=np.sum(counts)
    #beta=0.5
    outcomes_num = len(counts)
    # Compute hedged frequencies which are shifted to never be 0 or 1.
    freqs_hedged = (counts+beta)/(shots+outcomes_num*beta)
    # Return gaussian weights for 2-outcome measurements.
    return np.sqrt(shots/(freqs_hedged*(1-freqs_hedged)))

def _measurement_ops(label):#, meas_matrix_fn):
    #print(label)
    num_qubits=len(label)
    #print(num_qubits)
    meas_ops=[]
    for outcomes in sorted(it.product((0,1), repeat=num_qubits)):# (0,0,0,0,0)...(1,1,1,1,1)
        op=np.eye(1, dtype=complex)
        for m, outcome in zip(reversed(label), outcomes):#'X', 0...
            op=np.kron(op, measurement_matrix(m, outcome))
            #op=np.kron(op, meas_matrix_fn(m, outcome))
        meas_ops.append(op)
    return meas_ops

def _basis_operator_matrix(basis):
    num_ops=len(basis)
    nrows, ncols = basis[0].shape
    size = nrows * ncols
    ret = np.zeros((num_ops, size), dtype=complex)
    for j, b in enumerate(basis):
        ret[j] = np.array(b).reshape((1, size), order='F').conj()
    return ret

def preprocessing(result, basis_list=None):#result could be res_sim_M
    data={}
    #print(type(result))
    if(basis_list==None):
        res=result.results
        for i in range(243):
            data[res[i].header.name]=result.get_counts(i) 
    else:
        if(type(result)==dict):
            data = result
        else:
            for i in range(243):
                data[basis_list[i]]=result.get_counts(i)
    data_fit={}
    for basis in data.keys():
        tup=literal_eval(basis)
        counts=data[basis]
        if(data_fit.get(tup, 0)!=0):
            data_fit[tup]=combine_counts(data_fit[tup], counts)
        else:
            data_fit[tup]=counts
    return data_fit
#return: data_sim_fit

"""
fitter_data(data)
    Modified from [TomographyFitter's built-in method _fitter_data()](https://github.com/Qiskit/qiskit-ignis/blob/dd2c9265603329662f67a59a0e47744685b430d0/qiskit/ignis/verification/tomography/fitters/base_fitter.py)
    Variable: 
    Return:
        A tuple(fit_list, basis_blocks, weights)
        fit_list: list of int(no error mitigation) or float(with error mitigation) (length=243*32)
        basis_blocks: list of (32, 1024) ndarrays, length=243
        weights: list of float, length=243*32
"""
def fitter_data(data):#data_sim_fit
    #self._meas_basis!=None
    #measurement=meas_basis.measurement_matrix
    #self._prep_basis==None
    preparation=None
    fit_list=[]
    basis_blocks=[]
    weights=[]#standard_weights=True
    wts=0
    cts=0
    #Generate counts keys for converting to np array - is_qpt=false
    ctkeys=count_keys(5) #len(label), 5 for [[5,1,3]]

    for label, cts in data.items():
        # Convert counts dict to numpy array#only results with 00 outcomes are kept
        cts=np.array([cts.get('00 '+key, 0) for key in ctkeys])#'00 '+key, 0) for key in ctkeys])
        # Get probabilities
        shots=np.sum(cts)
        probs=np.array(cts)/shots
        fit_list+=list(probs)
        # Compute binomial weights - standard_weights is True
        wts=_binomial_weights(cts, 0.5)
        weights+=list(wts)
        # Get reconstruction basis operators - TBD
        prep_label=None
        meas_label=label
        #print(meas_label)
        prep_op=np.eye(1, dtype=complex)
        meas_ops=_measurement_ops(meas_label)#, measurement)

        block=_basis_operator_matrix([np.kron(prep_op.T, mop) for mop in meas_ops])
        basis_blocks.append(block)

    basis_blocks=np.vstack(basis_blocks)
    return fit_list, basis_blocks, weights

In [64]:
psi_qc_sim, qctom_sim, qctom_no_anc_sim, res_sim = sim_dict[0]

In [27]:
psi_qc_real, qctom_real, qctom_no_anc_real, res_real = real_dict[0]

In [138]:
psi_qc_real1, qctom_real1, qctom_no_anc_real1, res_real1 = real_dict[-1]

In [46]:
basis=[]
for i in range(243):
    basis.append(res_sim.results[i].header.name)

## Fidelity

In [55]:
"""
fidelity(data)
Variable
    data: A tuple (psi_qc, qctom, qctom_no_anc, res) for both simulation and experiment, should be 'dict[i](i=0, 1 or 2)' if the cells above are used.
    sf: syndrom & flag outcomes, '00', '01', '10', or '11'
Return:
    fidelity: a float number
"""
def fidelity(data, sf='00', backend_type='simulation', basis_list=None):
    psi_qc, qctom, qctom_no_anc, res= data
    if(backend_type=='simulation'):
        data_fit=preprocessing(res)
        fit_list, basis_blocks, weights=fitter_data(data_fit)
        #lstsq, result=(32, 32) ndarray
        result=lstsq_fit.lstsq_fit(fit_list, basis_blocks, weights=weights, psd=True, trace=1)
        return state_fidelity(result, reduced(psi_qc,sf))
    if(backend_type=='experiment'):
        data_fit=preprocessing(res, basis_list=basis_list)
        fit_list, basis_blocks, weights=fitter_data(data_fit)
        #lstsq, result=(32, 32) ndarray
        result=lstsq_fit.lstsq_fit(fit_list, basis_blocks, weights=weights, psd=True, trace=1)
        return state_fidelity(result, reduced(psi_qc,sf))

In [65]:
print(fidelity(sim_dict[0]))

0.9882463928449343


the first stabilizer with error mitigation

In [123]:
print(fidelity(real_dict[0], backend_type='experiment', basis_list=basis))

0.027345174904116407


20 identity operations instead of any stabilizer were applied onto the circuit after non-FTly preparing the |+> state. Error mitigation is used.

In [139]:
print(fidelity(real_dict[-1], backend_type='experiment', basis_list=basis))

0.02620737579775006


In [131]:
np.min(np.array([qctom_real[i].depth() for i in range(243)]))

24

Result: 

`res.data(i)`, $i\in 0..242$, returns {'counts':{'bitstring': count after error mitigation}}

'bitstring':7-bit bitstr, first two should be the syn&flag

counts are real numbers

`res.get_counts(i)`, return {'bitstring':count} but bitstring are still splited by ' '

In [341]:
#never used
"""
cnt_striped={}
for sf in ['00','01','10','11']:
    cnt_striped[sf]={}
    for i in range(243):
        cnt_striped[sf][i]={}
for i in range(243):
    for out, cnt in res_sim_M.data(i)['counts'].items():
        if(cnt>1e-7):
            tmp=''
            for i in range(int(cnt/10)+1):
                tmp+='-'
        #print(out, cnt)
        out1=format(int(key[2:], 16), "07b")
        cnt_striped[out1[:2]][i][out1[2:]]=cnt
print('Done')
"""

Done


### Analyze the first stabilizer's result

In [102]:
psi_qc0, results_sim0, result_sf0, tomo_qc_sim0, job_set_foo0, results_real0, tomo_qc_real0=dict[0]

In [105]:
# print(results_sim0.to_dict())
# print(result_sf0)
dummy = result_sf0['00'].dict()
# dummy = results_sim0.to_dict()
dimmy = deepcopy(dummy)

for idx, _ in enumerate(dimmy['results']):
    if (idx == 0):
        temp = _['data']['counts'] 
        print(temp)
#         print(type(temp))
        newcs = {}
        for idxx, j in enumerate(temp):
            print(type(j))
            k = str(hex(int(j,2)))
            newcs[k] = temp[j]
        _['data']['counts'] = newcs
        temp = _['data']['counts'] 
        print(temp)
print(dimmy)
res = Result.from_dict(dimmy)
# print(dimmy)
print(res)

{'0111010': 1.28229462980482e-12, '0100001': 4.206981564903849, '1010011': 2.553180706965399e-11, '0110110': 6.148322607781273e-13, '0001011': 244.1426789359314, '0011111': 217.85322362784106, '0100000': 1.5876238703698552e-11, '1000000': 1.655769446736303e-11, '0011100': 258.52353777711653, '1000110': 5.253763766267456e-12, '1000101': 1.7737533553069892e-12, '1110110': 1.174956661527179e-12, '0010011': 279.5670563129508, '1110100': 2.5822265376368693e-13, '1010110': 1.6903082615336717e-11, '1010100': 8.5014509316801e-12, '0110001': 1.0370631284808469e-11, '0111101': 7.523297494560635e-12, '0010101': 279.7549125798886, '0001101': 245.37263741988042, '0010111': 8.371876118307769e-11, '0011010': 245.60981580856114, '1000111': 1.0299752284440466e-11, '0111100': 2.0285597867977835e-11, '0101001': 5.6920719402032564e-12, '1100000': 1.1656154869325305e-12, '1001101': 1.1036509884338718e-11, '0011101': 9.469704987608183e-12, '0000011': 7.541051797530093e-11, '0001001': 4.8614913113569274e-11,

ModelValidationError: "{'results': {1: {'data': {'counts': [{'0100001': ['String does not match expected pattern.'], '1010011': ['String does not match expected pattern.'], '0001011': ['String does not match expected pattern.'], '0011111': ['String does not match expected pattern.'], '1100101': ['String does not match expected pattern.'], '1000010': ['String does not match expected pattern.'], '1111101': ['String does not match expected pattern.'], '0011100': ['String does not match expected pattern.'], '1101001': ['String does not match expected pattern.'], '1000101': ['String does not match expected pattern.'], '0010011': ['String does not match expected pattern.'], '1110100': ['String does not match expected pattern.'], '1010100': ['String does not match expected pattern.'], '0110001': ['String does not match expected pattern.'], '1001010': ['String does not match expected pattern.'], '0111101': ['String does not match expected pattern.'], '0010101': ['String does not match expected

In [42]:
qcc, qc0=StatePreparation(0)
qc0.draw()
qc1=deepcopy(qc0)
ca1 = ClassicalRegister(2)
qc1.add_register(ca1)
qc1.measure([qcc[5],qcc[6]],[ca1[0],ca1[1]])
job0 = execute(qc1, backend_sv)

In [43]:
qctom = tom.state_tomography_circuits(qc0, data_list, meas_labels='Pauli', meas_basis='Pauli')
qctom_no_anc = deepcopy(qctom)

for qc in qctom:
    ca = ClassicalRegister(2)
    qc.add_register(ca)
    qc.measure([qcc[5],qcc[6]],[ca[0],ca[1]])

In [ ]:
qctom1 = tom.state_tomography_circuits(qc0, [0,1,2,3,4,5,6], meas_labels='Pauli', meas_basis='Pauli')

In [44]:
qctom[-1].draw()

┌───┐         ┌───┐                      ░  ░ ┌─┐                  
   q7_0: ┤ H ├─■─────■─┤ X ├──────────────────────░──░─┤M├──────────────────
         ├───┤ │     │ └─┬─┘                      ░  ░ └╥┘┌─┐               
   q7_1: ┤ H ├─■──■──┼───┼────────■───────────────░──░──╫─┤M├───────────────
         ├───┤    │  │   │        │               ░  ░  ║ └╥┘┌─┐            
   q7_2: ┤ H ├─■──■──┼───┼────────┼───────────────░──░──╫──╫─┤M├────────────
         ├───┤ │     │   │        │               ░  ░  ║  ║ └╥┘┌─┐         
   q7_3: ┤ H ├─■──■──┼───┼────────┼───────────────░──░──╫──╫──╫─┤M├─────────
         ├───┤    │  │   │        │               ░  ░  ║  ║  ║ └╥┘┌─┐      
   q7_4: ┤ H ├────■──■───┼────────┼───────■───────░──░──╫──╫──╫──╫─┤M├──────
         ├───┤           │        │       │ ┌───┐ ░  ░  ║  ║  ║  ║ └╥┘┌─┐   
   q7_5: ┤ H ├───────────■────■───■───■───■─┤ H ├─░──░──╫──╫──╫──╫──╫─┤M├───
         └───┘              ┌─┴─┐   ┌─┴─┐   └───┘ ░  ░  ║  ║  ║  ║  ║ └╥┘┌─┐
   q7_6: ───────────────────┤ X ├───┤ X ├─────────░──░──╫──╫──╫──╫──╫──╫─┤M├
                            └───┘   └───┘         ░  ░  ║  ║  ║  ║  ║  ║ └╥┘
c1227_0: ═══════════════════════════════════════════════╩══╬══╬══╬══╬══╬══╬═
                                                           ║  ║  ║  ║  ║  ║ 
c1227_1: ══════════════════════════════════════════════════╩══╬══╬══╬══╬══╬═
                                                              ║  ║  ║  ║  ║ 
c1227_2: ═════════════════════════════════════════════════════╩══╬══╬══╬══╬═
                                                                 ║  ║  ║  ║ 
c1227_3: ════════════════════════════════════════════════════════╩══╬══╬══╬═
                                                                    ║  ║  ║ 
c1227_4: ═══════════════════════════════════════════════════════════╩══╬══╬═
                                                                       ║  ║ 
c1470_0: ══════════════════════════════════════════════════════════════╩══╬═
                                                                          ║ 
c1470_1: ═════════════════════════════════════════════════════════════════╩═

In [45]:
job = execute(qctom, backend=backend_sim, optimization_level=0, shots=4096)
results_sim=job.result()
print(results_sim)


Result(backend_name='ibmq_qasm_simulator', backend_version='0.4.1', date=datetime.datetime(2020, 5, 7, 3, 30, 1, 94198), header=Obj(backend_name='ibmq_qasm_simulator', backend_version='0.1.547'), job_id='5eb380ac69fc280011cc4e6e', metadata={'max_memory_mb': 257936, 'omp_enabled': True, 'parallel_experiments': 1, 'time_taken': 3.931323664}, qobj_id='8c3a60ae-fc8c-4aa8-a74e-cf12981b788d', results=[ExperimentResult(data=ExperimentResultData(counts=Obj(0x1=244, 0x10=268, 0x13=250, 0x15=262, 0x16=253, 0x19=275, 0x1a=243, 0x1c=250, 0x1f=252, 0x2=258, 0x4=258, 0x7=267, 0x8=255, 0xb=237, 0xd=274, 0xe=250)), header=Obj(clbit_labels=[['c1227', 0], ['c1227', 1], ['c1227', 2], ['c1227', 3], ['c1227', 4], ['c1228', 0], ['c1228', 1]], creg_sizes=[['c1227', 5], ['c1228', 2]], memory_slots=7, n_qubits=7, name="('X', 'X', 'X', 'X', 'X')", qreg_sizes=[['q7', 7]], qubit_labels=[['q7', 0], ['q7', 1], ['q7', 2], ['q7', 3], ['q7', 4], ['q7', 5], ['q7', 6]]), meas_level=<MeasLevel.CLASSIFIED: 2>, metadata={'

In [48]:
# TESTING

# Create an error mitigation filter ------
# ------used for testing measurement error mitigation on the simulator
measured_qubit_list = extractMsmtQubits(qctom[-1]) # assuming all the tomo circuits are identical except for basis changes
print(measured_qubit_list)
filt = msmtErrorMitFilter(backend, measured_qubit_list)
# Perform error mitigation on all the results
results_sim_MM = msmtErrorMitOnTom(results_sim, filt)
print(results_sim_MM)

[0, 1, 2, 3, 4, 5, 6]
{'00 00100': 258, '00 01101': 274, '00 10000': 268, '00 10110': 253, '00 01011': 237, '00 10101': 262, '00 01110': 250, '00 01000': 255, '00 11100': 250, '00 00001': 244, '00 11010': 243, '00 00010': 258, '00 11111': 252, '00 10011': 250, '00 11001': 275, '00 00111': 267}
{'0000100': 258, '0001101': 274, '0010000': 268, '0010110': 253, '0001011': 237, '0010101': 262, '0001110': 250, '0001000': 255, '0011100': 250, '0000001': 244, '0011010': 243, '0000010': 258, '0011111': 252, '0010011': 250, '0011001': 275, '0000111': 267}
{'0000000': 5.31761651963924e-14, '0000001': 225.4050810626695, '0000010': 238.59506261082794, '0000011': 3.160555077430494e-14, '0000100': 237.23162616801042, '0000101': 4.173760899851014e-14, '0000110': 3.375640246770188e-14, '0000111': 271.85193409185155, '0001000': 241.22109465643138, '0001001': 2.5978313547452105e-14, '0001010': 2.8320716511698977e-14, '0001011': 248.2955254108076, '0001100': 2.77120826433949e-14, '0001101': 290.1451214436

NameError: name 'new_counts2' is not defined

In [ ]:
job_tmp=job_set_foo0.jobs()[0]

In [ ]:
jobs_tmp=job_set_foo0.jobs()[0]

In [ ]:
numresult=0
for i in range(len(job_set_foo0.jobs())):
    numresult+=len(job_set_foo0.jobs()[i].result().get_counts())
print(numresult)

In [ ]:
result_tmp=result_sf0['00']

### Strip the registers of syndrome and flag in results obtained from real qc
only '00' is kept

In [ ]:
numjobs=len(job_set_foo0.jobs())
#sf only consider 00
new_real_results=[]
for jobidx in range(numjobs):
    #job_set_foo0.jobs()[jobidx]
    numresults=len(job_set_foo0.jobs()[jobidx].result().results)
    new_result = deepcopy(job_set_foo0.jobs()[jobidx].result())
    for resultidx in range(numresults):
        #outcomes: 00,01,10,11

        old_counts=job_set_foo0.jobs()[jobidx].result().get_counts()[resultidx]
        new_counts={}
        new_result.results[resultidx].header.creg_sizes = [new_result.results[resultidx].header.creg_sizes[0]]
        new_result.results[resultidx].header.clbit_labels = new_result.results[resultidx].header.clbit_labels[0:-2]
        new_result.results[resultidx].header.memory_slots = 5

        for reg_key in old_counts:
            reg_bits = reg_key.split(' ')
            if (reg_bits[0]=='00'):#reg_key is in the form like '00 10011', reg_bts[0]='00'
                new_counts[reg_bits[1]]=old_counts[reg_key]

        new_result.results[resultidx].data.counts = \
            new_result.results[resultidx]. \
            data.counts.from_dict(new_counts)

    new_real_results.append(new_result)


Since there are multiple jobs, we need to combine the results with all kinds of Pauli measurements together

In [ ]:
new_real_results_sum=new_real_results[0]

In [ ]:
for i in range(len(new_real_results)-1):
    new_real_results_sum.results.extend(new_real_results[i+1].results)

Syndrome and flag are always 0 in the ideal simulation 

In [181]:
rho_qc_sf0 = tomo_qc_sim0['11'].fit(method='lstsq')
# print(results_sim0)

NameError: name 'tomo_qc_sim0' is not defined

In [ ]:
print(results_sim0)

fidelity between the reduced statevector of the circuit without measurement, and the reconstructed state of simulation tomography results

In [182]:
F_qc_sf0 = state_fidelity(reduced(psi_qc0,'00'), rho_qc_sf0)
print(F_qc_sf0)

NameError: name 'psi_qc0' is not defined

The following fidelity calculations are incorrect because noisy experimental results include those with non-zero syndrome or flag.

The fidelity between reconstruction using the simulation data and reconstruction using the real qc data

In [183]:
tomo_qc_real00=tom.StateTomographyFitter(new_real_results_sum, qctom_no_anc, meas_basis='Pauli')

NameError: name 'new_real_results_sum' is not defined

Show the difference between simulation & experiment outcomes with different Pauli bases...(not very accurate)

$\sum_{Pauli} (\sum_{i=00000}^{11111} |prob(i, simulation)-prob(i, experiment)|)$

$Pauli=(XXXXX, XXXXY, ..., ZZZZZ)$


In [ ]:
sum_diff=0
for key in tomo_qc_sim0['00'].data.keys():
    sum_sim=np.sum(list(tomo_qc_sim0['00'].data[key].values()))
    sum_real=np.sum(list(tomo_qc_real00.data[key].values()))
    diff=0
    for i in range(2**5):
        diff+=np.abs((tomo_qc_sim0['00'].data[key].get(bin(i)[2:],0)/sum_sim - tomo_qc_real00.data[key].get(bin(i)[2:],0)/sum_real))
    print(key, diff)
    sum_diff+=diff
print(sum_diff)

In [ ]:
tomo_qc_real00.data.values()

In [ ]:
tomo_qc_sim0['00'].data

In [ ]:
tomo_qc_real00.data

In [ ]:
from qiskit.visualization import *

In [ ]:
plot_histogram([tomo_qc_real00.data[('Z', 'Z', 'Z', 'Z', 'Z')],tomo_qc_sim0['00'].data[('Z', 'Z', 'Z', 'Z', 'Z')]])

In [ ]:
F_qc_0=state_fidelity(rho_qc_sf0, rho_real_sf00)
print(F_qc_0)

Fidelity between the real-quantum-computer-data reconstruction and the ideal statevector

In [ ]:
F_qc_1=state_fidelity(reduced(psi_qc0,'00'), rho_real_sf00)
print(F_qc_1)

In [ ]:
test_allzero=np.zeros((32,32))
test_allzero[0][0]=1
F_qc_test=state_fidelity(reduced(psi_qc0, '00'),test_allzero)
print(F_qc_test)

In [ ]:
qctom_trans = transpile(qctom, backend=backend)

In [ ]:
for qc_trans in qctom_trans:
    print(qc_trans.depth())